In [775]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as pl

In [776]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [777]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [778]:
block_size = 3  
X, Y = [], []
for w in words:
  
  # print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    # print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] 
X = torch.tensor(X)
Y = torch.tensor(Y)

In [779]:
C = torch.randn((27, 2))


In [780]:
X[:3]

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13]])

In [781]:
C[torch.tensor([[0, 0, 0], 
                [0, 9, 5], 
                [0, 5, 13]])]

tensor([[[ 0.2250,  0.8549],
         [ 0.2250,  0.8549],
         [ 0.2250,  0.8549]],

        [[ 0.2250,  0.8549],
         [ 0.1622, -0.1713],
         [-0.9238,  0.3866]],

        [[ 0.2250,  0.8549],
         [-0.9238,  0.3866],
         [ 0.8207,  0.3444]]])

In [782]:
Dim_1 = F.one_hot(torch.tensor(1, ), num_classes=27).float()
Dim_1 @ C

tensor([ 0.7499, -0.0544])

In [783]:
Dim_1.shape

torch.Size([27])

In [784]:
C.shape

torch.Size([27, 2])

In [785]:
X.shape

torch.Size([228146, 3])

In [786]:
C[X].shape

torch.Size([228146, 3, 2])

In [787]:
C[X][5]

tensor([[0.2250, 0.8549],
        [0.2250, 0.8549],
        [0.2250, 0.8549]])

In [788]:
emb = C[X]

In [789]:
(emb[:, 0, :], emb[:, 1, :], emb[:, 2, :])

(tensor([[ 0.2250,  0.8549],
         [ 0.2250,  0.8549],
         [ 0.2250,  0.8549],
         ...,
         [ 0.4166, -1.8216],
         [ 0.4166, -1.8216],
         [-1.9329, -0.1605]]),
 tensor([[ 0.2250,  0.8549],
         [ 0.2250,  0.8549],
         [-0.9238,  0.3866],
         ...,
         [ 0.4166, -1.8216],
         [-1.9329, -0.1605],
         [ 0.4166, -1.8216]]),
 tensor([[ 0.2250,  0.8549],
         [-0.9238,  0.3866],
         [ 0.8207,  0.3444],
         ...,
         [-1.9329, -0.1605],
         [ 0.4166, -1.8216],
         [-1.0506,  2.0073]]))

In [790]:
emb.shape

torch.Size([228146, 3, 2])

In [791]:
W1 = torch.randn((6, 100))
B1 = torch.randn(100)

In [792]:
h = emb.view(-1, 6) @ W1 + B1

In [793]:
h = torch.tanh(h)
h.shape

torch.Size([228146, 100])

In [794]:
W2 = torch.randn((100, 27))
B2 = torch.randn(27)

logits = h @ W2 + B2 

In [795]:
count = logits.exp()

In [796]:
prob = count/count.sum(1, keepdim = True)

In [797]:
X.shape

torch.Size([228146, 3])

In [798]:
loss =  -prob[torch.arange(228146), Y].log().mean()
loss

tensor(16.7021)

In [799]:
parameters = sum([parameter.nelement() for parameter in [W1, W2, B1, B2, C]])
parameters

3481

In [800]:
loss = F.cross_entropy(logits, Y)

In [801]:
parameters = [parameter for parameter in [W1, W2, B1, B2, C]]

for parameter in parameters:
    parameter.requires_grad = True

In [802]:
[print(parameter.shape) for parameter in [W1, W2, B1, B2, C]]

torch.Size([6, 100])
torch.Size([100, 27])
torch.Size([100])
torch.Size([27])
torch.Size([27, 2])


[None, None, None, None, None]

In [803]:
for _ in range(10):
    emb = C[X]
    h = (emb.view(-1, 6) @ W1) + B1
    logits = h @ W2 + B2
    loss = F.cross_entropy(logits, Y)   
    for parameter in parameters:
        parameter.grad = None
    loss.backward()
    for p in parameters:
        p.data += -0.1*p.grad
print(loss.item()) 

12.076620101928711


In [809]:
for i in range(10):
  ix = torch.randint(0, X.shape[0], (32,))
  emb = C[X[ix]]
  h = torch.tanh(emb.view(-1, 6) @ W1 + B1) 
  logits = h @ W2 + B2 
  loss = F.cross_entropy(logits, Y[ix])
  for p in parameters:
    p.grad = None
  loss.backward()
  for p in parameters:

    print(p.grad)
    p.data += -0.1*p.grad

tensor([[-4.3710e-03,  3.2674e-03, -4.4445e-03,  3.4052e-03, -2.2006e-02,
          5.8948e-03, -1.1031e-02,  1.6844e-02, -6.8847e-03,  6.5872e-03,
         -1.5131e-03,  6.6706e-03, -3.7719e-03,  1.2456e-02, -1.5040e-02,
         -1.5339e-02, -4.4690e-03, -9.2919e-03, -1.1117e-02, -4.8837e-03,
         -8.5562e-04, -7.3944e-03,  5.4250e-03,  1.2777e-03,  1.3005e-03,
         -2.7264e-02, -8.5993e-04,  9.7450e-03,  9.0718e-03, -1.0248e-02,
          7.9196e-03,  1.8703e-02,  1.4915e-04,  2.2653e-03,  2.4052e-02,
          1.6290e-03, -9.6272e-04,  1.3268e-04,  3.3042e-03,  2.7933e-02,
          4.9119e-03, -8.7087e-03, -2.0016e-02, -3.7483e-02,  1.4486e-03,
          4.8161e-03,  2.9461e-03, -5.6295e-03,  1.2831e-02, -7.3198e-04,
         -6.0869e-03,  4.0533e-03, -3.0656e-02,  1.2444e-02, -3.0001e-05,
          2.6960e-02,  4.5526e-03, -1.9870e-02, -1.3361e-02, -3.0826e-03,
         -4.7283e-03, -1.6149e-03, -8.5956e-03,  1.9457e-02, -1.1360e-02,
         -4.3886e-03, -1.0813e-02, -1.

In [808]:
print(loss.item()) 

2.722496747970581


In [816]:
import random
block_size = 3  
def build_dataset(words):  
  X, Y = [], []
  for w in words: 
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix) 
      context = context[1:] + [ix] 

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [817]:
g = torch.Generator().manual_seed(2147483647)  
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [821]:
for p in parameters:
  p.requires_grad = True

In [818]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [819]:

lri = []
lossi = []
stepi = []


In [822]:

for i in range(200000):
  ix = torch.randint(0, Xtr.shape[0], (32,))

  emb = C[Xtr[ix]] # (32, 3, 10)
  h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 200)
  logits = h @ W2 + b2 # (32, 27)
  loss = F.cross_entropy(logits, Ytr[ix])
  
  for p in parameters:
    p.grad = None
  loss.backward()
  
  lr = 0.1 if i < 100000 else 0.01
  for p in parameters:
    p.data += -lr * p.grad
  stepi.append(i)
  lossi.append(loss.log10().item())


In [823]:
emb = C[Xtr]  
h = torch.tanh(emb.view(-1, 30) @ W1 + b1)  
logits = h @ W2 + b2  
loss = F.cross_entropy(logits, Ytr)
loss

tensor(2.1204, grad_fn=<NllLossBackward0>)

In [825]:
emb = C[Xdev] 
h = torch.tanh(emb.view(-1, 30) @ W1 + b1) 
logits = h @ W2 + b2  
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.1603, grad_fn=<NllLossBackward0>)

In [826]:
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size  
    while True:
      emb = C[torch.tensor([context])] 
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

moha.
mayah.
see.
med.
ryla.
renvrettagrarg.
aderediel.
izemi.
jen.
eden.
estanarielle.
malailin.
shus.
rishiriel.
jenne.
jennex.
berlyn.
brey.
dariyah.
fael.
